In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
text = str(input(""))
df_test = pd.DataFrame([text],columns =['sentence'])
df_test.head()

,sentence
0,its an amazing day great to have you as a partner


In [3]:
# General defination of preprocessing for test data
import unicodedata
import string
import textblob
import nltk
import contractions
import re

# remove some stopwords to capture negation in n-grams if possible
stop_words = nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

# load up a simple porter stemmer - nothing fancy
ps = nltk.porter.PorterStemmer()

class Preprocess():
    def remove_accented_chars(text):
        a = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return a

    def simple_text_preprocessor(document): 
        # lower case
        document = str(document).lower()
        
        # expand contractions
        document = contractions.fix(document)
        
        # remove unnecessary characters
        document = re.sub(r'[^a-zA-Z]',r' ', document)
        document = re.sub(r'nbsp', r'', document)
        document = re.sub(' +', ' ', document)
        
        # simple porter stemming
        document = ' '.join([ps.stem(word) for word in document.split()])
        
        # stopwords removal
        document = ' '.join([word for word in document.split() if word not in stop_words])
        
        return document


In [4]:
df = pd.read_csv("/Users/rishitkapoor/Documents/Python_ML/NLP/sentiment_train.csv")
a = np.array(df['sentence'])
li =[]
for i in a:
    li.append(Preprocess.remove_accented_chars(i))

df['sentence']=[x for x in li]

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['sentence']], df[['label']],random_state=1,test_size=0.3,stratify=df[['label']])

In [6]:
import string

X_train['char_count'] = X_train['sentence'].apply(len)
X_train['word_count'] = X_train['sentence'].apply(lambda x: len(x.split()))
X_train['word_density'] = X_train['char_count'] / (X_train['word_count']+1)
X_train['punctuation_count'] = X_train['sentence'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_train['title_word_count'] = X_train['sentence'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count'] = X_train['sentence'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

X_test['char_count'] = X_test['sentence'].apply(len)
X_test['word_count'] = X_test['sentence'].apply(lambda x: len(x.split()))
X_test['word_density'] = X_test['char_count'] / (X_test['word_count']+1)
X_test['punctuation_count'] = X_test['sentence'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_test['title_word_count'] = X_test['sentence'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count'] = X_test['sentence'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [7]:
import textblob
x_train_snt_obj = X_train['sentence'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_train['Polarity'] = [obj.polarity for obj in x_train_snt_obj.values]
X_train['Subjectivity'] = [obj.subjectivity for obj in x_train_snt_obj.values]

x_test_snt_obj = X_test['sentence'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_test['Polarity'] = [obj.polarity for obj in x_test_snt_obj.values]
X_test['Subjectivity'] = [obj.subjectivity for obj in x_test_snt_obj.values]


In [8]:

stp = np.vectorize(Preprocess.simple_text_preprocessor)
X_train['Clean sentence'] = stp(X_train['sentence'].values)
X_test['Clean sentence'] = stp(X_test['sentence'].values)

In [9]:
X_train_metadata = X_train.drop(['sentence', 'Clean sentence'], axis=1).reset_index(drop=True)
X_test_metadata = X_test.drop(['sentence', 'Clean sentence'], axis=1).reset_index(drop=True)


CountVectorizer is a great tool provided by the scikit-learn library in Python. It is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text.



In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer as tf_idf
tf = tf_idf(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = tf.fit_transform(X_train['Clean sentence']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=tf.get_feature_names())

X_testcv = tf.transform(X_test['Clean sentence']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=tf.get_feature_names())
X_traincv.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aaron,abandon,abbey,abort,abram,absolut,absurd,academi,accent,accept,...,yip,young,younger,youth,yupanqui,zach,zeal,zen,zesti,zoot
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
'''from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))
X_traincv = cv.fit_transform(X_train['Clean sentence']).toarray()
X_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())

X_testcv = cv.transform(X_test['Clean sentence']).toarray()
X_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names())
X_traincv.head()'''

"from sklearn.feature_extraction.text import CountVectorizer\n\ncv = CountVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1, 1))\nX_traincv = cv.fit_transform(X_train['Clean sentence']).toarray()\nX_traincv = pd.DataFrame(X_traincv, columns=cv.get_feature_names())\n\nX_testcv = cv.transform(X_test['Clean sentence']).toarray()\nX_testcv = pd.DataFrame(X_testcv, columns=cv.get_feature_names())\nX_traincv.head()"

In [12]:
X_train_comb = pd.concat([X_train_metadata, X_traincv], axis=1)
X_test_comb = pd.concat([X_test_metadata, X_testcv], axis=1)

X_train_comb.head()

,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,aaron,abandon,...,yip,young,younger,youth,yupanqui,zach,zeal,zen,zesti,zoot
0,40,5,6.666667,2,2,0,-1.000000,1.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,35,5,5.833333,3,0,0,0.166667,1.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,132,27,4.714286,9,11,1,0.500000,0.60000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,35,4,7.000000,3,1,0,0.850000,1.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,126,25,4.846154,6,5,4,0.368750,0.74375,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
c = list(df_test.iloc[:,0])
df_test['sentence']=[Preprocess.remove_accented_chars(c[0])]
X_test_new = df_test[['sentence']]

X_test_new['char_count'] = X_test_new['sentence'].apply(len)
X_test_new['word_count'] = X_test_new['sentence'].apply(lambda x: len(x.split()))
X_test_new['word_density'] = X_test_new['char_count'] / (X_test_new['word_count']+1)
X_test_new['punctuation_count'] = X_test_new['sentence'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
X_test_new['title_word_count'] = X_test_new['sentence'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test_new['upper_case_word_count'] = X_test_new['sentence'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

x_test_snt_obj = X_test_new['sentence'].apply(lambda row: textblob.TextBlob(row).sentiment)
X_test_new['Polarity'] = [obj.polarity for obj in x_test_snt_obj.values]
X_test_new['Subjectivity'] = [obj.subjectivity for obj in x_test_snt_obj.values]

X_test_new['Clean sentence'] = stp(X_test_new['sentence'].values)

X_test_metadata_2 = X_test_new.drop(['sentence', 'Clean sentence'], axis=1).reset_index(drop=True)

X_testcv_2 = tf.transform(X_test_new['Clean sentence']).toarray()
X_testcv_2 = pd.DataFrame(X_testcv_2, columns=tf.get_feature_names())

X_test_comb_2 = pd.concat([X_test_metadata_2, X_testcv_2], axis=1)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1, random_state=1, solver='lbfgs',multi_class='ovr')
lr.fit(X_train_comb, y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1, multi_class='ovr', random_state=1)

In [15]:
'''predictions = lr.predict(X_test_comb)
print(classification_report(y_test, predictions))
pd.DataFrame(confusion_matrix(y_test, predictions))'''

'predictions = lr.predict(X_test_comb)\nprint(classification_report(y_test, predictions))\npd.DataFrame(confusion_matrix(y_test, predictions))'

In [16]:
pred_new = lr.predict(X_test_comb_2)
proba = "{:.2f}".format((lr.predict_proba(X_test_comb_2).max())*100)
cs = np.array(pred_new)
if (cs[0]==1):
    print(str(proba)+'% Positive')
else:
    print(str(proba)+'% Negative')

99.99% Positive
